# 编码器—解码器（encoder-decoder）

在基于词语的语言模型中，我们使用了循环神经网络。它的输入是一段不定长的序列，输出却是定长的，例如一个词语。然而，很多问题的输出也是不定长的序列。以机器翻译为例，输入是可以是英语的一段话，输出可以是法语的一段话，输入和输出皆不定长，例如

    英语：They are watching.

    法语：Ils regardent.



当输入输出都是不定长序列时，我们可以使用编码器—解码器（encoder-decoder）或者seq2seq

编码器（encoder）对应输入序列，解码器（decoder）对应输出序列。下面我们来介绍编码器—解码器的设计。

![encoder-decoder](./images/encoder_decoder.png)

编码器的作用是把一个不定长的输入序列转化成一个定长的 Context 向量 $(\mathbf{c})$ 。该向量包含了输入序列的信息。

解码器的作用是接收一个定长的 Context 向量 $(\mathbf{c})$ 。然后解读该向量得到一个不定长的输出序列。


常用的编码器、解码器是循环神经网络，但是需要注意的是：这时候的循环神经网络通常会是多层的。


## 编码器（Encoder）

为了演示Encoder，我们构建一个比较简单的编码器 Encoder，一个 Embedding 层和一个GRU层 （这里我们不使用已经训练好的词向量，而是直接使用 nn.Embedding，在实际项目中最好使用 Glove 或者 word2vec）

![encoder-network](./images/encoder-network.png)


需要注意的是，在我们演示的模型中，我们选择 encoder 最后一个 hidden layer 的输出作为 encoder 的输出，这个输出有时候也被称为 *context vector* 因为，它可以被看作是将整个序列的信息进行编码而得到的。


## 解码器（Decoder）

为了演示Decoder，我们构建一个比较简单的编码器 Decoder，也是一个 Embedding 层和一个GRU层

![decoder-network](./images/decoder-network.png)


ps: 上述图像来自 http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html


## 大纲

为了实现一个 Neural Translation Machine 我们需要按照以下步骤行动：

1、

2、

3、



## 第一步：构建一个Config类，用于保存各种超参数，以及导入各种包

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import unicodedata, string, re, random, time, math

In [2]:
class Config():
    def __init__(self):
        self.data_path = "../data/cmn-eng/cmn.txt"
        self.use_gpu = True
        self.hidden_size = 256
        self.encoder_lr = 1e-5
        self.decoder_lr = 1e-5
        self.train_num = 75000 # 训练数据集的数目
        self.print_epoch = 5000
config = Config()

## 第二步：数据预处理

准备数据的全部过程如下所示：

1. 读取txt文件，并按行分割，再把每一行分割成一个pair (Eng, Chinese)
2. 过滤并处理文本信息
3. 从每个pair中，制作出 中文词典 和 英文词典
4. 构建训练集

data下载地址为： http://www.manythings.org/anki/cmn-eng.zip

该数据集中还有其他类型的翻译数据 http://www.manythings.org/anki/

中文词典和英文词典，我们使用*Lang* 类，该类包含了所有的 中文（英文） -> 数字 或者 数字 -> 中文（英文）的映射。

同时，我们要给一句话的其实和结束加上标志符

起始符：(Start Of Sentence)

SOS_token = 0

结束符：(End Of Sentence)

EOS_token = 1

另外，在这个类中，我们需要添加一个 *word2count* 方法，用来计算各个词出现的次数




In [3]:
SOS_token = 0
EOS_token = 1

class Lang():
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.word2count = {}
        self.n_words = 2  # Count SOS and EOS
    
    def addSentence(self, sentence):
        if self.name == "Chinese":
            for word in sentence:
                self.addWord(word)
        else:
            for word in sentence.split(' '):
                self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, pairs_file, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(pairs_file, encoding='utf-8').read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = []
    for l in lines:
        temp = l.split('\t')
        eng_unit = normalizeString(temp[0])
        chinese_unit = temp[1]
        pairs.append([eng_unit, chinese_unit])
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 15  # 长度大于15的我们统统舍弃

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ",
    "i", "he", 'you', 'she', 'we',
    'they', 'it'
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1]) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
def prepareData(lang1, lang2, pairs_file, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, pairs_file, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, "字典的大小为", str(input_lang.n_words))
    print(output_lang.name, "字典的大小为", str(output_lang.n_words))
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('Eng', 'Chinese', config.data_path)
print(random.choice(pairs))

Reading lines...
Read 19777 sentence pairs
Trimmed to 9473 sentence pairs
Counting words...
Counted words:
Eng 字典的大小为 3737
Chinese 字典的大小为 2638
['you should always tell the truth .', '你應該永遠說實話。']


**到目前为止，我们已经把字典构建好了，接下来就是构建训练集**

In [10]:
def indexesFromSentence(lang, sentence):
    if lang.name == "Chinese":
        return [lang.word2index[word] for word in sentence]
    else:
        return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence, use_gpu):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1)) # seq*1
    if use_gpu:
        return result.cuda()
    else:
        return result

def variablesFromPair(pair, use_gpu):
    input_variable = variableFromSentence(input_lang, pair[0], use_gpu)
    target_variable = variableFromSentence(output_lang, pair[1], use_gpu)
    return (input_variable, target_variable)

In [11]:
# 随机获取2个训练数据集， 这里我们依旧不用进行 batch 处理，下一章节 attention 机制中，我们再进行 batch 处理
example_pairs = [variablesFromPair(random.choice(pairs), config.use_gpu)
                      for i in range(2)]
print(example_pairs)

[(Variable containing:
    4
   87
   40
  352
  854
 1636
    6
    1
[torch.cuda.LongTensor of size 8x1 (GPU 0)]
, Variable containing:
    6
  130
  131
   21
  271
  500
  137
 1143
  592
    4
    1
[torch.cuda.LongTensor of size 11x1 (GPU 0)]
), (Variable containing:
   22
   23
  198
  125
   38
   23
   37
    6
    1
[torch.cuda.LongTensor of size 9x1 (GPU 0)]
, Variable containing:
  297
  356
  265
   47
   25
   26
    9
    4
    1
[torch.cuda.LongTensor of size 9x1 (GPU 0)]
)]


## 第三步：构建编码器

编码器的结构，如图所示：


![encoder-network](./images/encoder-network.png)



In [33]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x).view(1, x.size()[0], -1)
        output = embedded  # batch*seq*feature
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self, use_gpu):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_gpu:
            return result.cuda()
        else:
            return result

## 第四步：构建解码器

编码器的结构，如图所示：

![decoder-network](./images/decoder-network.png)


In [40]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, x, hidden):
        output = self.embedding(x).view(1, 1, -1)
        output = F.relu(output)  
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, use_gpu):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_gpu:
            return result.cuda()
        else:
            return result

## 第五步：开始训练

定义优化器、损失函数，然后开始进行训练


In [41]:
# 实例化模型

encoder = Encoder(input_lang.n_words, config.hidden_size)
encoder = encoder.cuda() if config.use_gpu else encoder

decoder = Decoder(config.hidden_size, input_lang.n_words)
decoder = decoder.cuda() if config.use_gpu else decoder

# 定义优化器

encoder_optimizer = optim.Adam(encoder.parameters(), lr=config.encoder_lr)

decoder_optimizer = optim.Adam(decoder.parameters(), lr=config.decoder_lr)


# 定义损失函数

fn_loss = nn.NLLLoss()

training_pairs = [variablesFromPair(random.choice(pairs), config.use_gpu)
                      for i in range(config.train_num)]

In [42]:
# 开始训练
for iter in range(1, config.train_num+1):
    training_pair = training_pairs[iter - 1]
    input_variable = training_pair[0]  # seq_len * 1
    target_variable = training_pair[1]  # seq_len * 1
    
    loss = 0
    
    # 训练过程
    encoder_hidden = encoder.initHidden(config.use_gpu)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    
    # 传入 encoder
    encoder_output, encoder_hidden = encoder(input_variable, encoder_hidden)
    
    # decoder 起始
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if config.use_gpu else decoder_input
    
    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)          
        targ = target_variable[di]
        loss += fn_loss(decoder_output, targ)
        decoder_input = targ
    
    # 反向求导
    loss.backward()
    # 更新梯度
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    print_loss = loss.data[0] / target_length
    
    if iter % config.print_epoch == 0:
        print("loss is: %.4f" % (print_loss))

KeyboardInterrupt: 

## 第六步：随机采样，对模型进行测试


